This notebook is about to test the futbin.csv data file with neural network models. This notebook is running in the Google colab environment to use the computational capacity of the cloud.
The dataset is the same futbin.csv but uploaded into Google drive.

In [1]:
try:
    import scikeras
except ImportError:
    !python -m pip install scikeras

In [2]:
!python -m pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.0 MB/s eta 0:00:00


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow_addons.metrics import RSquare
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
%matplotlib inline

Mount google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!ls -la "/content/drive/My Drive/"

ls: '/content/drive/My Drive/termékek_Hegyalja út': No such file or directory
total 4442
-rw------- 1 root root    2722 Jul 17  2022  all.txt
drwx------ 2 root root    4096 Oct 17  2019 'foci tippek'
-rw------- 1 root root      30 Jul 17  2022  GB.txt
drwx------ 2 root root    4096 Aug  1  2020  Hegyalja
-rw------- 1 root root     177 Jun 26  2023 'LJMU Timeline.gsheet'
drwx------ 2 root root    4096 Aug 27 10:55 'Meet Recordings'
-rw------- 1 root root 1477507 Jan 11  2023  passport.pdf
-rw------- 1 root root 1284466 Oct  8  2022  passport.png
drwx------ 2 root root    4096 Jul 27  2021 'Picture share'
drwx------ 2 root root    4096 May 16  2023  Print
drwx------ 2 root root    4096 Jul 15  2022  python
-rw------- 1 root root   17807 Nov 21  2022 'Qatar VB táblázat 1 forduló+ pont.xlsx'
-rw------- 1 root root 1499011 Apr 13  2023 'SBS-058 klímacső.rtf'
-rw------- 1 root root     177 Apr 13  2023 'SBS-058 klímacső.rtf.gdoc'
-rw------- 1 root root     177 Sep 29  2022  Signatur

This notebook is used the following page as a source:
https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/


https://machinelearningmastery.com/regression-metrics-for-machine-learning/

https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/

https://machinelearningmastery.com/train-final-machine-learning-model/

https://pyimagesearch.com/2019/01/21/regression-with-keras/

**Part 1**: Import the futbin data and do feature transformations, scaling

In [6]:
df=pd.read_csv('/content/drive/My Drive/Upgrad/Thesis/3. Thesis/jupyter/futbin.csv')
df.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,...,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing
0,Pelé,98,3270000.0,5,4,95.0,96,93,96,60,...,0,0,0,0,0,0,0,0,0,1
1,Lionel Messi,98,4350000.0,4,4,93.0,98,97,99,40,...,0,0,0,0,0,0,0,0,0,1
2,Lionel Messi,98,4640000.0,4,4,94.0,97,96,99,40,...,0,0,0,0,0,0,0,0,0,1
3,Karim Benzema,97,1850000.0,4,5,92.0,97,90,94,45,...,0,0,0,0,0,0,0,0,0,1
4,Kylian Mbappé,97,9750000.0,5,4,99.0,96,88,98,44,...,0,0,0,0,0,0,0,0,0,1


In [7]:
X = df.copy()
# Remove names
names = X.pop('Name')
# Remove target
Y = X.pop('Price')

# perform a scaler transform of the input data
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

# perform log transformation of target variable
#Y = np.log(Y)

In [8]:
#df_scaled = pd.DataFrame(X)
#df_scaled.head(2)
X.head(2)

,Rating,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,...,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing
0,98,5,4,95.0,96,93,96,60,76,3928,...,0,0,0,0,0,0,0,0,0,1
1,98,4,4,93.0,98,97,99,40,77,5575,...,0,0,0,0,0,0,0,0,0,1


In [9]:
Y

0       3270000.0
1       4350000.0
2       4640000.0
3       1850000.0
4       9750000.0
          ...    
6392        200.0
6393        200.0
6394        200.0
6395        200.0
6396        250.0
Name: Price, Length: 6397, dtype: float64

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.1, random_state=42)

**Part 2**: Create Model Using keras

First simple model, it has an input layer, one middle layer and one output layer.

In [ ]:
# define base model
def baseline_model():
  model = keras.Sequential([
      keras.layers.Dense(103, activation=tf.nn.relu, input_shape=(103,)),
      keras.layers.Dense(64, activation=tf.nn.relu),
      keras.layers.Dense(1, kernel_initializer='normal')])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
              loss='mean_absolute_error',
              metrics=['accuracy'])
  return model

In [ ]:
%%time
estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_absolute_error')
print("Baseline: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Baseline: -25587.38 (73517.96) MAE
CPU times: user 58min 1s, sys: 4min 5s, total: 1h 2min 6s
Wall time: 52min 32s


In [ ]:
results

array([-246121.4970661 ,   -4001.22066154,    -810.84492302,
          -604.93874817,    -681.073874  ,    -586.9273036 ,
          -630.75519876,    -762.28534762,    -968.3951171 ,
          -705.87007917])

The second model is a deeper structure, it has more middle layers.

In [13]:
# define deeper model
def deeper_model():
  model = keras.Sequential([
    keras.layers.Dense(103, activation=tf.nn.relu, input_shape=(103,)),
    keras.layers.Dense(96, activation=tf.nn.relu),
    keras.layers.Dense(82, activation=tf.nn.relu),
    keras.layers.Dense(68, activation=tf.nn.relu),
    keras.layers.Dense(42, activation=tf.nn.relu),
    keras.layers.Dense(30, activation=tf.nn.relu),
    keras.layers.Dense(22, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(8, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.relu),
    keras.layers.Dense(1, kernel_initializer='normal')])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
             loss='mean_absolute_error',
              metrics=['accuracy'])
  return model

In [ ]:
%%time
estimator2 = KerasRegressor(model=deeper_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator2, X, Y, cv=kfold, scoring='neg_mean_absolute_error')
print("Deeperline: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Deeperline: -25351.55 (73550.37) MAE
CPU times: user 1h 13min 16s, sys: 2min 5s, total: 1h 15min 21s
Wall time: 1h 1min 30s


The third model is a wider model, the input layer is wider than the input size.

In [ ]:
# define wider model
def wider_model():
  model = keras.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu, input_shape=(103,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(8, activation=tf.nn.relu),
    keras.layers.Dense(1, kernel_initializer='normal')])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
             loss='mean_absolute_error',
              metrics=['accuracy'])
  return model

In [ ]:
%%time
estimator3 = KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator3, X, Y, cv=kfold, scoring='neg_mean_absolute_error')
print("Widerline: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Widerline: -25352.51 (73550.02) MAE
CPU times: user 1h 37s, sys: 2min 4s, total: 1h 2min 42s
Wall time: 53min 57s


Deeper model with standardized step

In [12]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=deeper_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

In [ ]:
%%time
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_absolute_error')
print("Standardized deeper: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Standardized deeper: -25352.17 (73550.25) MAE
CPU times: user 1h 11min 24s, sys: 2min 1s, total: 1h 13min 26s
Wall time: 1h 5min 12s


In [ ]:
estimators2 = []
estimators2.append(('standardize', StandardScaler()))
estimators2.append(('mlp', KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline2 = Pipeline(estimators2)

In [ ]:
%%time
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline2, X, Y, cv=kfold, scoring='neg_mean_absolute_error')
print("Standardized wider: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Standardized wider: -25354.22 (73549.22) MAE
CPU times: user 1h 6min 50s, sys: 2min 10s, total: 1h 9min
Wall time: 1h 2min 6s


Final model validation
The deeper model is the best, predict y with it

In [35]:
# define final model
def final_model():
  model = keras.Sequential([
    keras.layers.Dense(103, activation=tf.nn.relu, input_shape=(103,)),
    keras.layers.Dense(96, activation=tf.nn.relu),
    keras.layers.Dense(82, activation=tf.nn.relu),
    keras.layers.Dense(68, activation=tf.nn.relu),
    keras.layers.Dense(42, activation=tf.nn.relu),
    keras.layers.Dense(30, activation=tf.nn.relu),
    keras.layers.Dense(22, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(8, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.relu),
    keras.layers.Dense(1, activation="linear", kernel_initializer='normal')])

  opt = Adam(learning_rate=1e-3)
  model.compile(optimizer=opt,
             loss='mean_absolute_percentage_error')
  return model

In [36]:
final_model = final_model()

In [37]:
final_model.fit(x=X_train, y=y_train,
	validation_data=(X_val, y_val),
	epochs=500, batch_size=20)

Epoch 1/500
288/288 [==============================] - 5s 6ms/step - loss: 51.4495 - val_loss: 44.8032
Epoch 2/500
288/288 [==============================] - 2s 6ms/step - loss: 42.6928 - val_loss: 45.8522
Epoch 3/500
288/288 [==============================] - 2s 7ms/step - loss: 42.4587 - val_loss: 44.7754
Epoch 4/500
288/288 [==============================] - 2s 8ms/step - loss: 42.2166 - val_loss: 45.2641
Epoch 5/500
288/288 [==============================] - 2s 7ms/step - loss: 42.5118 - val_loss: 44.6385
Epoch 6/500
288/288 [==============================] - 2s 6ms/step - loss: 42.0538 - val_loss: 45.0621
Epoch 7/500
288/288 [==============================] - 2s 6ms/step - loss: 41.9767 - val_loss: 44.1036
Epoch 8/500
288/288 [==============================] - 2s 6ms/step - loss: 41.8099 - val_loss: 44.5602
Epoch 9/500
288/288 [==============================] - 2s 5ms/step - loss: 41.5748 - val_loss: 43.9130
Epoch 10/500
288/288 [==============================] - 2s 5ms/step - los

In [38]:
preds = final_model.predict(X_val)

20/20 [==============================] - 0s 2ms/step


In [39]:
diff = preds.flatten() - y_val
percentDiff = (diff / y_val) * 100
absPercentDiff = np.abs(percentDiff)

In [40]:
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

In [41]:
print("[INFO] avg. player price: {}, std player price: {}".format(
	df["Price"].mean(), df["Price"].std()))

[INFO] avg. player price: 25574.98045959043, std player price: 328078.5808615929


In [42]:
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] mean: 34.23%, std: 35.77%
